In [23]:
param_2D = Dict{String,Any}()
param_2D["n_plac"]= 100
param_2D["temperature"] = 1.
param_2D["lenght"] = 1.
param_2D["d_rest"] = [0.2, 0.4]
param_2D["elasticity"] = [1., 1.]
param_2D["step"]= [0.1, 0.1, 0.1]
param_2D

Dict{String,Any} with 6 entries:
  "step"        => [0.1, 0.1, 0.1]
  "elasticity"  => [1.0, 1.0]
  "lenght"      => 1.0
  "n_plac"      => 100
  "temperature" => 1.0
  "d_rest"      => [0.2, 0.4]

In [7]:
function energy_2D(x, y, param_2D) 
    link_1 = x[1:2] - y[1:2] + param_2D["lenght"] *([cospi(x[3]), sinpi(x[3])] - [cospi(y[3]), sinpi(y[3])])
    link_2 = x[1:2] - y[1:2] 
    energy_1 = param_2D["elasticity"][1]*(sqrt(sum(link_1.^2)) - param_2D["d_rest"][1])^2
    energy_2 = param_2D["elasticity"][2]*(sqrt(sum(link_2.^2)) - param_2D["d_rest"][2])^2
    return energy_1 + energy_2
end

energy_2D (generic function with 1 method)

In [11]:
energy_2D([0.,0.,0.], [0.,1.,0.], param_2D) 

1.0

In [38]:
chain = rand( 10, 3)

10×3 Array{Float64,2}:
 0.263029  0.41564   0.465265
 0.862179  0.699714  0.0720943
 0.593279  0.653728  0.528353
 0.548059  0.404528  0.500913
 0.651997  0.1505    0.417383
 0.688795  0.902852  0.0476367
 0.475308  0.030019  0.375512
 0.492569  0.145191  0.726406
 0.735597  0.37085   0.673018
 0.31643   0.423654  0.326708

In [26]:
chain[2, :]

4-element Array{Float64,1}:
 0.4265888466908456
 0.6363956895423233
 0.7714836519761119
 0.3982212022586691

In [16]:
using Random
function element_progress_2D(chain, elem, param_2D)
    energy_before = energy_2D(chain[elem,:], chain[elem-1,:], param_2D) + energy_2D(chain[elem,:], chain[elem+1,:], param_2D)
    var = (rand(3).-0.5).*param_2D["step"]
    energy_after = energy_2D(chain[elem,:] + var, chain[elem-1,:], param_2D) + energy_2D(chain[elem,:] + var, chain[elem+1,:], param_2D)
    delta_energy = energy_after - energy_before
    if delta_energy < 0. || rand() > exp(- delta_energy/param_2D["temperature"])
        chain[elem, :] += var
    #else println("non ho cambiato")
    end
end

element_progress_2D (generic function with 1 method)

In [25]:
chain = fill(0., param_2D["n_plac"], 3)
chain[1,3] = 0.5
chain[param_2D["n_plac"],:] = [1, 0, 0.5]
for j = 1:1000
    for i = 2:(param_2D["n_plac"]-1)
        element_progress_2D(chain, i, param_2D)
    end
end
chain

100×3 Array{Float64,2}:
  0.0          0.0         0.5
  0.329328    -0.084086    0.5596
 -0.121941    -0.0924047   0.354862
 -0.315981     0.277196    0.162449
  0.0382053    0.119392    0.225998
 -0.0240213    0.544024    0.0348185
 -0.150873     0.195709    0.0980041
  0.0817145   -0.098263    0.155919
 -0.166695     0.218299   -0.0111462
 -0.327337    -0.137623    0.0918597
  0.00520853  -0.346078    0.187055
  0.0263431    0.0706957   0.0226508
 -0.0368435   -0.318846    0.0899264
  ⋮                       
  0.0259496    0.0723941   0.157521
  0.126039    -0.309863    0.220453
  0.151805     0.107524    0.106182
 -0.0436383    0.460017   -0.0453891
  0.0562416    0.0972495   0.146131
  0.0896648   -0.304415    0.266756
  0.0450221    0.113046    0.193351
  0.333938    -0.178753    0.377514
  0.103123     0.169373    0.230801
  0.224105    -0.181875    0.322354
  0.636254    -0.141079    0.41269
  1.0          0.0         0.5

In [21]:
element_progress_2D(chain, 4, 0.1)
element_progress_2D(chain, 4, 0.1)
element_progress_2D(chain, 4, 0.1)
chain

LoadError: [91mMethodError: no method matching getindex(::Float64, ::String)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  getindex(::Number) at number.jl:75[39m
[91m[0m  getindex(::Number, [91m::Integer[39m) at number.jl:76[39m
[91m[0m  getindex(::Number, [91m::Integer...[39m) at number.jl:81[39m
[91m[0m  ...[39m

In [82]:
sum([0.5, 2].*[3, 1.1])

3.7

In [34]:
function azzera_3(chain)
    chain[3,:] = [0., 0., 0.]
end
chain_1 = fill(1, 10,3)
azzera_3(chain_1)
chain_1

10×3 Array{Int64,2}:
 1  1  1
 1  1  1
 0  0  0
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

In [9]:
map(x-> sinpi(x), [1, 2, 3, 4])

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [116]:
D = Dict([("A", [1, 3]), ("B", 3)])

Dict{String,Any} with 2 entries:
  "B" => 3
  "A" => [1, 3]

In [119]:
D["C"] = [2, 1, 3.]
D["D"] = 19.
D

Dict{String,Any} with 4 entries:
  "B" => 3
  "A" => [1, 3]
  "C" => [2.0, 1.0, 3.0]
  "D" => 19.0